### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [90]:
# Adding "pass_math" and "pass_reading" columns to the complete DataFrame for use in later steps. 
school_data_complete["pass_math"] = ["pass" if x >= 70 else "fail" for x in school_data_complete["math_score"]]
school_data_complete["pass_reading"] = ["pass" if x >= 70 else "fail" for x in school_data_complete["reading_score"]]


# Creating a list that contains all of the unique "school_name" entries in the complete data set. 
# Setting the length of the list equal to a variable to get a count of the number of schools. 
number_schools = len(school_data_complete["school_name"].unique())


# Creating a list that contains all of the unique "student_name" entries in the complete data set. 
# Setting the length of the list equal to a variable to get a count of the number of students. 
number_students = len(school_data_complete["Student ID"].unique())


# Creating a list that contains all of the unique "budget" entries in the complete data set. 
# Setting the sume of the list equal to a variable to combine the budgets of all schools. 
total_budget = sum(school_data_complete["budget"].unique())


# Calculating the mean of all math scores in the dataset and setting the value equal to a variable. 
average_math = school_data_complete["math_score"].mean()

# Calculating the mean of all reading scores in the dataset and setting the value equal to a variable. 
average_reading = school_data_complete["reading_score"].mean()


# Creating a DataFrame consisting of only the students whose entry for "pass_math" was "pass".
# Setting a variable passing_math equal to the number of entries in the Series "Student ID".
passing_math_df = school_data_complete.loc[school_data_complete["pass_math"] == "pass", ["Student ID","school_name", "math_score"]]
passing_math = passing_math_df["Student ID"].count()


# Setting variable equal to the number of students passing math divided by the number of total students, multiplied by 100. 
percent_math_district = (passing_math/number_students) * 100


# Creating a DataFrame consisting of only the students whose entry for "pass_reading" was "pass".
# Setting a variable passing_math equal to the number of entries in the Series "Student ID".
passing_reading_df = school_data_complete.loc[school_data_complete["pass_reading"] == "pass", ["Student ID","school_name", "reading_score"]]
passing_reading = passing_reading_df["Student ID"].count()


# Setting variable equal to the number of students passing reading divided by the number of total students, multiplied by 100. 
percent_reading_district = (passing_reading/number_students) * 100


# Creating a DataFrame consisting of only the students whose entries for for "pass_math" and "pass_reading" were both "pass".
# Setting a variable passing_math equal to the number of entries in the Series "Student ID".
passing_overall_df = school_data_complete.loc[(school_data_complete["pass_math"] == "pass") & (school_data_complete["pass_reading"] == "pass"), ["Student ID","school_name", "math_score", "reading_score"]]
passing_overall = passing_overall_df["Student ID"].count()


# Setting variable equal to the number of students passing overall divided by the number of total students, multiplied by 100. 
percent_overall_district = (passing_overall/number_students) * 100


# Creating an indexed DataFrame that sets each of the variables above as the value for the associated key. 
district_summary_df = pd.DataFrame(
    {"Total Schools": number_schools, "Total Students": number_students, "Total Budget": total_budget, "Average Math Score": average_math, "Average Reading Score": average_reading, "% Passing Math": percent_math_district, "% Passing Reading": percent_reading_district, "% Passing Overall": percent_overall_district}, index = [0])

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [94]:
# Creating a DataFrame using .loc that contains the desired values from the complete dataset. 
# Note: the DataFrame does not contain type because, had it been included, we would not have been able to calculate the mean of the included values (you can't perform the mean function on a string).
school_data_short = school_data_complete.loc[:,["school_name", "size", "budget", "math_score", "reading_score"]]


# Grouping the above DataFrame by "school_name". 
# Calculating mean of the included values (the budget and size remain the same and we get the mean of the math and reading scores).
school_data_grouped = school_data_short.groupby(["school_name"])
school_data_mean = school_data_grouped.mean()


# Assigning a variable to each column in the above groupby object. 
mean_reading = school_data_mean["reading_score"]
mean_math = school_data_mean["math_score"]
mean_size = school_data_mean["size"]
mean_budget = school_data_mean["budget"]


# Creating a DataFrame containing the varaibles above (already grouped by school name in earlier step).
school_data_mean_df = pd.DataFrame({"Total Students": mean_size, "Total School Budget": mean_budget, "Average Math Score": mean_math, "Average Reading Score": mean_reading})


# Grouping previously created DataFrame (passing_math_df) by "school_name" and setting equal to a variable.
# Counting the number of entries (i.e., the number of students passing math) for each school and setting it equal to a variable. 
# Adding a % Passing Math Series to the running DataFrame, defined as the number of students passing math for each school divided by the size of each school, multiplied by 100. 
pass_math_group = passing_math_df.groupby(["school_name"])
pass_math_count = pass_math_group.count()
school_data_mean_df["% Passing Math"] = (pass_math_count["math_score"]/school_data_mean_df["Total Students"]) * 100


# Performing the same operation as above for % Passing Reading. 
pass_reading_group = passing_reading_df.groupby(["school_name"])
pass_reading_count = pass_reading_group.count()
school_data_mean_df["% Passing Reading"] = (pass_reading_count["reading_score"]/school_data_mean_df["Total Students"]) * 100


#Performing the same operation as above for % Passing Overall. 
pass_overall_group = passing_overall_df.groupby(["school_name"])
pass_overall_count = pass_overall_group.count()
school_data_mean_df["% Passing Overall"] = (pass_overall_count["reading_score"]/school_data_mean_df["Total Students"]) * 100


# Creating a DataFrame containing the "school_name" and "type" from the complete data set.
# Removing duplicates from the school type DataFrame. 
school_type_complete = school_data_complete.loc[:,["school_name","type"]] 
school_type_short = school_type_complete.drop_duplicates()


# Merging the school type DataFrame with the mean values DataFrame on "school_name".
schools_merged = pd.merge(school_data_mean_df, school_type_short, on = "school_name", how = "inner")


# Adding a budget "Per Student" column to the merged DataFrame, calculated by dividing the budget Series by the size Series. 
schools_merged["Per Student Budget"] = schools_merged["Total School Budget"]/schools_merged["Total Students"]


# Re-ording and re-indexing merged DataFrame into new, cleaner DataFrame.
school_summary_df = pd.DataFrame({"": schools_merged["school_name"], "School Type": schools_merged["type"], "Total Students": schools_merged["Total Students"], "Total School Budget": schools_merged["Total School Budget"], "Per Student Budget": schools_merged["Per Student Budget"], "Average Math Score": schools_merged["Average Math Score"], "Average Reading Score": schools_merged["Average Reading Score"], "% Passing Math": schools_merged["% Passing Math"], "% Passing Reading": schools_merged["% Passing Reading"], "% Passing Overall": schools_merged["% Passing Overall"] })
school_summary = school_summary_df.set_index("")

school_summary




,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,,,,,,,,,
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [95]:
# Using iloc to create a new DataFrame containing the top five schools, indexed by % Passing Overall, from the school_summary DataFrame. 
Top_5 = school_summary.sort_values("% Passing Overall", ascending = False).iloc[:5]

Top_5



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,,,,,,,,,
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [29]:
# Using iloc to create a new DataFrame containing the bottom five schools, indexed by % Passing Overall, from the school_summary DataFrame. 
Bottom_5 = school_summary.sort_values("% Passing Overall").iloc[:5]

Bottom_5



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,,,,,,,,,
Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [96]:
# Creating a DataFrame consisting of the "school_name" and "math_score" Series from the complete data set for all rows whose "grade" entry is "9th". 
# Grouping the ninth DataFrame by "school_name". 
# Setting a variable equal to the mean of the groupby object. 
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name", "math_score"]]
ninth_grouped = ninth.groupby(["school_name"])
ninth_mean = ninth_grouped.mean()


# Performing the aboce operation for 10th grade. 
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name","math_score"]]
tenth_grouped = tenth.groupby(["school_name"])
tenth_mean = tenth_grouped.mean()


# Performing the above operation for 11th grade.
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name", "math_score"]]
eleventh_grouped = eleventh.groupby(["school_name"])
eleventh_mean = eleventh_grouped.mean()


# Performing the above operation for 12th grade. 
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name", "math_score"]]
twelfth_grouped = twelfth.groupby(["school_name"])
twelfth_mean = twelfth_grouped.mean()

# Creating a DataFrame containing the "math_score" columns from each of the mean groupby objects. 
math_grades = pd.DataFrame({"9th": ninth_mean["math_score"],"10th": tenth_mean["math_score"], "11th": eleventh_mean["math_score"], "12th": twelfth_mean["math_score"] })

math_grades





,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [97]:
# Creating a DataFrame consisting of the "school_name" and "reading_score" Series from the complete data set for all rows whose "grade" entry is "9th". 
# Grouping the ninth DataFrame by "school_name". 
# Setting a variable equal to the mean of the groupby object. 
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name", "reading_score"]]
ninth_grouped = ninth.groupby(["school_name"])
ninth_mean = ninth_grouped.mean()


# Performing the aboce operation for 10th grade. 
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name","reading_score"]]
tenth_grouped = tenth.groupby(["school_name"])
tenth_mean = tenth_grouped.mean()


# Performing the above operation for 11th grade.
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name", "reading_score"]]
eleventh_grouped = eleventh.groupby(["school_name"])
eleventh_mean = eleventh_grouped.mean()


# Performing the above operation for 12th grade. 
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name", "reading_score"]]
twelfth_grouped = twelfth.groupby(["school_name"])
twelfth_mean = twelfth_grouped.mean()


# Creating a DataFrame containing the "reading_score" columns from each of the mean groupby objects. 
reading_grades = pd.DataFrame({"9th": ninth_mean["reading_score"],"10th": tenth_mean["reading_score"], "11th": eleventh_mean["reading_score"], "12th": twelfth_mean["reading_score"] })

reading_grades



,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [98]:
# Using .iloc and an existing DataFrame to create a new DataFrame with the desired Series. 
school_spending = school_summary_df.iloc[:, 4:10]


# Binning data accoriding to spending ranges. 
bins = [0, 584.99, 629.99, 644.99, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]
school_spending["Spending Ranges (Per Student)"] = pd.cut(school_spending["Per Student Budget"], bins, labels = group_names, include_lowest = True)


# Deleting unneeded Series. 
del school_spending["Per Student Budget"]


# Grouping data by bins and calculating the mean values of each bin. 
# Storing this DataFrame as a variable. 
spending_ranges = school_spending.groupby(["Spending Ranges (Per Student)"])
spending_ranges_mean = spending_ranges.mean()

spending_ranges_mean




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [99]:
# Using .loc and an existing DataFrame to create a new DataFrame with the desired Series.
school_size = school_summary_df.loc[:, ["Total Students", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"]]


# Binning data by school size ranges. 
bins = [0, 999, 1999, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size["School Size"] = pd.cut(school_summary_df["Total Students"], bins, labels = group_names, include_lowest = True)


# Deleting unneeded Series. 
del school_size["Total Students"]


# Grouping data by bins and calculating the mean values of each bin. 
# Storing this DataFrame as a variable. 
size_ranges = school_size.groupby(["School Size"])
size_ranges_mean = size_ranges.mean()

size_ranges_mean



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [100]:
# Using .loc and an existing DataFrame to create a new DataFrame with the desired Series.
school_type = school_summary_df.loc[:, ["School Type", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"]]


# Grouping data by bins and calculating the mean values of each bin. 
# Storing this DataFrame as a varaible. 
type_groups = school_type.groupby(["School Type"])
type_groups_mean = type_groups.mean()

type_groups_mean




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [ ]:
# Two Observable Trends:
 
# 1. Based on the data, it appears that charter schools have higher overall passing percentages than district schools. 
#    Indeed, within our sample, students attending charter schools scored higher on all measures of performance (i.e., average scores and passing percentages), for both math and reading, than those attending district schools.  

# 2. The data also suggests that students attending larger schools perform worse than those attending medium or small schools, across all measures of performance. 
#    This same trend was not observed when comparing only small and medium schools. 
